In [1]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import math
import datetime
import tqdm
import pprint

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

#load the mnist dataset
(train_ds, test_ds) = tfds.load('cifar10', split=['train', 'test'], as_supervised=True)


c:\Users\clara\anaconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def prepare_data(batch_size, data): 
  '''
  prepare the dataset to have one-hot-vectors and values between -1 and 1
  '''
  data = data.map(lambda img, target: (tf.cast(img, tf.float32), tf.cast(target,tf.int32)))
  data = data.map(lambda img, target: ((img/128.)-1.0, target))
  data = data.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  
  data = data.cache()
  data = data.shuffle(1000)
  data = data.batch(batch_size)
  data = data.prefetch(20)

  return data


In [3]:
from tensorflow.keras.layers import Dense

class BasicConv(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 32 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConv, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [5]:
from tensorflow.keras.layers import Dense

class BasicConvFilter1(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 64 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvFilter1, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [4]:
from tensorflow.keras.layers import Dense

class BasicConvFilter2(tf.keras.Model):
  '''
  create a small network with 2 blocks each having 2 layers and starting with 16 filters in the first layer
  '''
  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvFilter2, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [14]:
from tensorflow.keras.layers import Dense

class BasicConvDeep(tf.keras.Model):
  '''
  create a deeper network with 4 blocks each having 2 layers and starting with 32 filters in the first layer
  '''

  def __init__(self,optimizer = tf.keras.optimizers.Adam()):
    super(BasicConvDeep, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #original input size is 32*32, we have 32 as number of filters
    self.l1 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    #after pooling the size is reduced to 16*16
    self.l3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l4 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.l5 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l6 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.pooling2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.l7 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')
    self.l8 = tf.keras.layers.Conv2D(filters = 128, kernel_size = 3, padding = 'same', activation = 'relu')

    self.global_pool = tf.keras.layers.GlobalAvgPool2D()

    # last layer is a dense layer with softmax activation 
    self.out = tf.keras.layers.Dense(10, activation = 'softmax')

  def call(self, x): 
    x = self.l1(x)
    x = self.l2(x)
    x = self.pooling(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.pooling1(x)
    x = self.l5(x)
    x = self.l6(x)
    x = self.pooling2(x)
    x = self.l7(x)
    x = self.l8(x)
    x = self.global_pool(x)
    x = self.out(x)

    return x


  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

In [4]:
class DenselyConnectedCNNLayer(tf.keras.layers.Layer):
  def __init__(self, num_filter): 
    super(DenselyConnectedCNNLayer, self).__init__()

    self.conv = tf.keras.layers.Conv2D(filters = num_filter, kernel_size = 3, padding = "same", activation = "relu")

  def call(self,x): 
    c = self.conv(x)
    #concatenate at last layer - get long stack of all earlier layer
    x = tf.concat((x,c), axis = -1)
    return x


class DenselyConnectedCNNBlock(tf.keras.layers.Layer):
  #take all conv layers and create a block we take for one convolution
  def __init__(self, num_filters, layers):
    super(DenselyConnectedCNNBlock, self).__init__()
    self.layers = [DenselyConnectedCNNLayer(num_filters) for _ in range(layers)]

  def call(self, x): 
    #take input and pump it through all layers we have 
    for l in self.layers:
      x = l(x)
      return x


class DenselyConnectedCNN(tf.keras.Model):
  def __init__(self, optimizer = tf.keras.optimizers.Adam()):
    super(DenselyConnectedCNN, self).__init__()

    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]

    #4 layers with each of 24 kernels -> get 4 layer in 28*28 resolution 
    self.denseblock1 = DenselyConnectedCNNBlock(24,4)
    #go down to 14*14
    self.pooling1 = tf.keras.layers.MaxPooling2D()

    self.denseblock2 = DenselyConnectedCNNBlock(24,4)
    self.pooling2 = tf.keras.layers.MaxPooling2D()

    self.denseblock3 = DenselyConnectedCNNBlock(24,4)
    self.globalpooling = tf.keras.layers.GlobalAvgPool2D()
    self.out = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.denseblock1(x)
    x = self.pooling1(x)
    x = self.denseblock2(x)
    x = self.pooling2(x)
    x = self.denseblock3(x)
    x = self.globalpooling(x)
    x = self.out(x)
    return x

  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}


dense_model = DenselyConnectedCNN()


In [ ]:
class ResidualConnectedCNNLayer(tf.keras.layers.Layer):
  def __init__(self, num_filters, optimizer = tf.keras.optimizers.Adam()):
    super(ResidualConnectedCNNLayer, self).__init__()
    self.conv = tf.keras.layers.Conv2D(filters=num_filters, kernel_size=3, padding='same', activation='relu')

  def call(self, x):
    c = self.conv(x)
    #residual connection 
    x = c+x
    return x

class ResidualConnectedCNNBlock(tf.keras.layers.Layer):
  def __init__(self, depth, layers):
    super(ResidualConnectedCNNBlock, self).__init__()
    #depth increases in the end of the network (cannot add them with different sizes)
    #specify the depth such that they get the same depth 
    self.deeper_layer = tf.keras.layers.Conv2D(filters=depth, kernel_size=3, padding='same', activation='relu')
    #list with many residual connected cnn layer
    self.layers = [ResidualConnectedCNNLayer(depth) for _ in range(layers)]

  def call(self, x):
    #bring it to the correct depth 
    x = self.deeper_layer(x)
    for layer in self.layers:
      x = layer(x)
    return x


class ResidualConnectedCNN(tf.keras.Model):
  def __init__(self, optimizer = tf.keras.optimizers.Adam()):
    super(ResidualConnectedCNN, self).__init__()
    self.optimizer = optimizer
    self.loss_function = tf.keras.losses.CategoricalCrossentropy()

    self.metrics_list = [ tf.keras.metrics.Mean(name="loss"),
                        tf.keras.metrics.CategoricalAccuracy(name="acc"),
    ]
    self.residualblock1 = ResidualConnectedCNNBlock(24,4)
    self.pooling1 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.residualblock2 = ResidualConnectedCNNBlock(48,4)
    self.pooling2 = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)

    self.residualblock3 = ResidualConnectedCNNBlock(96,4)
    self.globalpooling = tf.keras.layers.GlobalAvgPool2D()
    
    self.out = tf.keras.layers.Dense(10, activation='softmax')

  def call(self,x):
    x = self.residualblock1(x)
    x = self.pooling1(x)
    x = self.residualblock2(x)
    x = self.pooling2(x)
    x = self.residualblock3(x)
    x = self.globalpooling(x)
    x = self.out(x)
    return x

  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    x, t = data
    #print(x.shape)

    with tf.GradientTape() as tape:
      output = self(x, training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    x, t = data

    output = self(x, training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}


resnet = ResidualConnectedCNN()

In [5]:
def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer): 
  '''
  create the training loop where the model is trained on the train dataset and afterwards tested on the validation dataset
  '''
  #iterate over given amount of epochs
  for epoch in range(epochs): 
    print(f"Epoch {epoch}: ")

    #train on all batches of the training data
    for data in tqdm.tqdm(train_ds, position = 0, leave = True):

      metrics = model.train_step(data)


      with train_summary_writer.as_default(): 
        for metric in model.metrics: 
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

    # print the metrics
    print([f"Train {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric 
    model.reset_metrics()
  
    #evaluation on validation set
    for data in val_ds:
      metrics = model.test_step(data)

      with val_summary_writer.as_default():
        for metric in model.metrics:
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
      
    # print the metrics
    print([f"Test {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric
    model.reset_metric()

    print("\n")

In [6]:
def create_summary_writers(config_name):
  '''
  create the summary writer to have access to the metrics of the model 
  '''
  current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  train_log_path = f"logs/submission/{config_name}/{current_time}/train"
  val_log_path = f"logs/submission/{config_name}/{current_time}/val"

  # log writer
  train_summary_writer = tf.summary.create_file_writer(train_log_path)
  val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
  return train_summary_writer, val_summary_writer

In [7]:
import matplotlib.pyplot as plt 
def run(model,num_epochs=15,save=True,load=False,config_name = f'RUN'):
  '''
  run the model and specify the number of epochs, batch_size and prepare the training and testing data
  '''

  ### Hyperparameters
  num_epochs = num_epochs
  batch_size = 32
  train = prepare_data(batch_size,train_ds)
  test = prepare_data(batch_size,test_ds)
  
  train_summary_writer, val_summary_writer = create_summary_writers(config_name)
  if load:
    model.load_weights(f"saved_model_{config_name}");

  training_loop(model,train,test,num_epochs,train_summary_writer, val_summary_writer)
  if save:
    model.save_weights(f"saved_model_{config_name}", save_format="tf")

  

In [9]:
myMod1 = BasicConv(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
print("BasicConv, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod1,num_epochs=5,config_name=f'BASIC1')






BasicConv, optimizer=SGD, learning_rate=0.001, momentum=0.9

Epoch 0: 


1563it [00:40, 38.37it/s]


['Train loss: 2.2134850025177', 'Train acc: 0.16502000391483307']
['Test loss: 2.0349600315093994', 'Test acc: 0.24869999289512634']


Epoch 1: 


1563it [00:38, 40.57it/s]


['Train loss: 1.9514833688735962', 'Train acc: 0.26660001277923584']
['Test loss: 1.848431944847107', 'Test acc: 0.30140000581741333']


Epoch 2: 


...

['Train loss: 1.7756378650665283', 'Train acc: 0.32429999113082886']
['Test loss: 1.6967555284500122', 'Test acc: 0.3619999885559082']


Epoch 3: 


1563it [00:41, 37.97it/s]


['Train loss: 1.6891220808029175', 'Train acc: 0.36406001448631287']
['Test loss: 1.6489534378051758', 'Test acc: 0.3817000091075897']


Epoch 4: 


1563it [00:43, 36.29it/s]


['Train loss: 1.64059579372406', 'Train acc: 0.383899986743927']
['Test loss: 1.6171492338180542', 'Test acc: 0.3919000029563904']




In [9]:
myMod1 = BasicConv(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
run(myMod1,num_epochs=1,load=True,config_name=f'BASIC1')

Epoch 0: 


1563it [00:46, 33.68it/s]


['Train loss: 1.1866061687469482', 'Train acc: 0.5818399786949158']
['Test loss: 1.1931204795837402', 'Test acc: 0.5712000131607056']




In [11]:
myMod2 = BasicConv(tf.keras.optimizers.Adam(learning_rate=0.001))
print("\nBasicConv, optimizer=Adam, learning_rate=0.001\n")
run(myMod2,num_epochs=10,config_name=f'BASIC2')




BasicConv, optimizer=Adam, learning_rate=0.001

Epoch 0: 


1563it [00:44, 35.50it/s]


['Train loss: 1.6375659704208374', 'Train acc: 0.3905799984931946']
['Test loss: 1.3847239017486572', 'Test acc: 0.5031999945640564']


Epoch 1: 


1563it [00:41, 37.47it/s]


['Train loss: 1.273235559463501', 'Train acc: 0.5435400009155273']
['Test loss: 1.1705288887023926', 'Test acc: 0.5873000025749207']


Epoch 2: 


...

['Train loss: 0.8053713440895081', 'Train acc: 0.7185400128364563']
['Test loss: 0.8017879128456116', 'Test acc: 0.7197999954223633']


Epoch 8: 


1563it [00:40, 39.01it/s]


['Train loss: 0.7610055208206177', 'Train acc: 0.734499990940094']
['Test loss: 0.7891982197761536', 'Test acc: 0.7258999943733215']


Epoch 9: 


1563it [00:43, 35.64it/s]


['Train loss: 0.7312162518501282', 'Train acc: 0.7436800003051758']
['Test loss: 0.7528896927833557', 'Test acc: 0.7400000095367432']




In [12]:
print("saved after epoch 10: ")
run(myMod2,num_epochs=10,load=True,config_name=f'BASIC2')

saved after epoch 10: 
Epoch 0: 


1563it [00:58, 26.77it/s]


['Train loss: 0.6973021030426025', 'Train acc: 0.7575399875640869']
['Test loss: 0.7385714054107666', 'Test acc: 0.7465999722480774']


Epoch 1: 


1563it [01:02, 25.08it/s]


['Train loss: 0.6687029004096985', 'Train acc: 0.7667999863624573']
['Test loss: 0.7558719515800476', 'Test acc: 0.7401000261306763']


Epoch 2: 


...

['Train loss: 0.5360148549079895', 'Train acc: 0.8131600022315979']
['Test loss: 0.6976509690284729', 'Test acc: 0.765500009059906']


Epoch 8: 


1563it [00:42, 36.41it/s]


['Train loss: 0.5162753462791443', 'Train acc: 0.819819986820221']
['Test loss: 0.7367110848426819', 'Test acc: 0.763700008392334']


Epoch 9: 


1563it [00:44, 35.11it/s]


['Train loss: 0.49874794483184814', 'Train acc: 0.8247799873352051']
['Test loss: 0.7247179746627808', 'Test acc: 0.7627999782562256']




In [19]:
myMod3 = BasicConv(tf.keras.optimizers.RMSprop(learning_rate=0.001))
print("BasicConv, optimizer=RMSProp, learning_rate=0.001\n")
run(myMod3,num_epochs=2,save=False)


BasicConv, optimizer=RMSProp, learning_rate=0.001

Epoch 0: 


1563it [01:05, 23.91it/s]


['Train loss: 1.716761589050293', 'Train acc: 0.3566800057888031']
['Test loss: 1.4790164232254028', 'Test acc: 0.45660001039505005']


Epoch 1: 


1563it [01:02, 25.18it/s]


['Train loss: 1.4093519449234009', 'Train acc: 0.49215999245643616']
['Test loss: 1.4270565509796143', 'Test acc: 0.4853000044822693']




In [ ]:
myMod4 = BasicConv(tf.keras.optimizers.Adagrad(learning_rate=0.001))
print("\nBasicConv, optimizer=Adagrad, learning_rate=0.001\n")
run(myMod4,num_epochs=5,save=False)


In [ ]:
myMod5 = BasicConv(tf.keras.optimizers.SGD(learning_rate=0.001))

print("\BasicConv, optimizer=SGD, learning_rate=0.001\n")
run(myMod5,num_epochs=5,save=False)

In [ ]:
myMod6 = BasicConvFilter1(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod7 = BasicConvFilter1(tf.keras.optimizers.Adam(learning_rate=0.001))

print("BasicConvFilter1 with a higher number of filters, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod6,num_epochs=5,save=False)
print("\nBasicConvFilter1 with a higher number of filters, optimizer=Adam, learning_rate=0.001\n")
run(myMod7,num_epochs=5,save=False)

In [10]:
#myMod8 = BasicConvFilter2(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod9 = BasicConvFilter2(tf.keras.optimizers.Adam(learning_rate=0.001))

print("BasicConvFilter2 with a lower number of filters, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(myMod9,num_epochs=10,config_name=f'BASICFILTER2_ADAM')
# print("\nBasicConvFilter2 with a lower number of filters, optimizer=Adam, learning_rate=0.001\n")
# run(myMod9)

BasicConvFilter2 with a lower number of filters, optimizer=SGD, learning_rate=0.001, momentum=0.9

Epoch 0: 


1563it [00:40, 38.81it/s]


['Train loss: 1.7304245233535767', 'Train acc: 0.35016000270843506']
['Test loss: 1.5244892835617065', 'Test acc: 0.4334000051021576']


Epoch 1: 


1563it [00:38, 40.54it/s]


['Train loss: 1.4499415159225464', 'Train acc: 0.47648000717163086']
['Test loss: 1.3735796213150024', 'Test acc: 0.49540001153945923']


Epoch 2: 


...

['Train loss: 1.043208360671997', 'Train acc: 0.6312800049781799']
['Test loss: 1.0550329685211182', 'Test acc: 0.6230999827384949']


Epoch 8: 


1563it [00:19, 80.02it/s]


['Train loss: 1.0117554664611816', 'Train acc: 0.6416000127792358']
['Test loss: 0.9958971738815308', 'Test acc: 0.6431999802589417']


Epoch 9: 


1563it [00:20, 76.70it/s]


['Train loss: 0.9886159300804138', 'Train acc: 0.6494200229644775']
['Test loss: 0.9773367643356323', 'Test acc: 0.6486999988555908']




In [12]:
myMod9 = BasicConvFilter2(tf.keras.optimizers.Adam(learning_rate=0.001))

print("saved after epoch 15")
run(myMod9,num_epochs=10,load=True,config_name=f'BASICFILTER2_ADAM')

saved after epoch 15
Epoch 0: 


1563it [00:25, 61.50it/s]


['Train loss: 0.8706306219100952', 'Train acc: 0.693120002746582']
['Test loss: 0.8881240487098694', 'Test acc: 0.6865000128746033']


Epoch 1: 


1563it [00:21, 71.72it/s]


['Train loss: 0.8599962592124939', 'Train acc: 0.6975399851799011']
['Test loss: 0.8869524598121643', 'Test acc: 0.6880000233650208']


Epoch 2: 


...

['Train loss: 0.789778470993042', 'Train acc: 0.7226999998092651']
['Test loss: 0.8385594487190247', 'Test acc: 0.7069000005722046']


Epoch 8: 


1563it [00:20, 75.71it/s]


['Train loss: 0.7765979766845703', 'Train acc: 0.7283999919891357']
['Test loss: 0.8523053526878357', 'Test acc: 0.6991999745368958']


Epoch 9: 


1563it [00:21, 74.01it/s]


['Train loss: 0.7722440361976624', 'Train acc: 0.7299000024795532']
['Test loss: 0.8375826478004456', 'Test acc: 0.7074999809265137']




In [15]:
#myMod10 = BasicConvDeep(tf.keras.optimizers.SGD(learning_rate=0.001, momentum = 0.9))
myMod11 = BasicConvDeep(tf.keras.optimizers.Adam(learning_rate=0.001))

# print("BasicConvDeep with a deeper network, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
# run(myMod10)
print("\nBasicConvDeep with a deeper network, optimizer=Adam, learning_rate=0.001\n")
run(myMod11,num_epochs=10,config_name=f'BASICDEEP_ADAM')


BasicConvDeep with a deeper network, optimizer=Adam, learning_rate=0.001

Epoch 0: 


1563it [01:07, 23.04it/s]


['Train loss: 1.5317734479904175', 'Train acc: 0.42552000284194946']
['Test loss: 1.1670392751693726', 'Test acc: 0.5759999752044678']


Epoch 1: 


1563it [01:21, 19.17it/s]


['Train loss: 0.9884729981422424', 'Train acc: 0.646619975566864']
['Test loss: 0.9020010232925415', 'Test acc: 0.6818000078201294']


Epoch 2: 


...

['Train loss: 0.3260960876941681', 'Train acc: 0.8835200071334839']
['Test loss: 0.861922025680542', 'Test acc: 0.767799973487854']


Epoch 8: 


1563it [01:11, 21.88it/s]


['Train loss: 0.27957257628440857', 'Train acc: 0.9010800123214722']
['Test loss: 0.8676460385322571', 'Test acc: 0.7732999920845032']


Epoch 9: 


1563it [01:13, 21.39it/s]


['Train loss: 0.2396579533815384', 'Train acc: 0.9145799875259399']
['Test loss: 0.9578555226325989', 'Test acc: 0.7767999768257141']




In [16]:
print("after 10 epochs")
run(myMod11,num_epochs=5,load=True,config_name=f'BASICDEEP_ADAM')

after 10 epochs
Epoch 0: 


1563it [01:08, 22.94it/s]


['Train loss: 0.2092805653810501', 'Train acc: 0.9253000020980835']
['Test loss: 1.0070730447769165', 'Test acc: 0.7756999731063843']


Epoch 1: 


1563it [01:06, 23.39it/s]


['Train loss: 0.18894332647323608', 'Train acc: 0.9341599941253662']
['Test loss: 1.0846259593963623', 'Test acc: 0.7796000242233276']


Epoch 2: 


...

['Train loss: 0.1796649545431137', 'Train acc: 0.9375200271606445']
['Test loss: 1.058705449104309', 'Test acc: 0.7832000255584717']


Epoch 3: 


1563it [01:09, 22.45it/s]


['Train loss: 0.16614237427711487', 'Train acc: 0.94132000207901']
['Test loss: 1.001442313194275', 'Test acc: 0.7712000012397766']


Epoch 4: 


1563it [01:09, 22.51it/s]


['Train loss: 0.15666665136814117', 'Train acc: 0.9456200003623962']
['Test loss: 1.1458014249801636', 'Test acc: 0.7754999995231628']




In [ ]:
myMod12 = ResidualConnectedCNN(tf.keras.optimizers.SGD(learning_rate=0.001))

print("Residual Convolutional Network, optimizer=SGD, learning_rate=0.001")
run(myMod12,num_epochs=5,save=False)

In [9]:
myMod13 = DenselyConnectedCNN(tf.keras.optimizers.SGD(learning_rate=0.001))

print("Densely Connected Convolutional Network, optimizer=SGD\n")
run(myMod13,num_epochs=5)



Densely Connected Convolutional Network, optimizer=SGD

Epoch 0: 


100%|██████████| 1563/1563 [01:34<00:00, 16.52it/s]


['Train loss: 2.292332410812378', 'Train acc: 0.12451999634504318']
['Test loss: 2.264366626739502', 'Test acc: 0.1712000072002411']


Epoch 1: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.245129108428955', 'Train acc: 0.18014000356197357']
['Test loss: 2.218503952026367', 'Test acc: 0.19349999725818634']


Epoch 2: 


...

['Train loss: 2.2026288509368896', 'Train acc: 0.19562000036239624']
['Test loss: 2.175246477127075', 'Test acc: 0.20980000495910645']


Epoch 3: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.162001609802246', 'Train acc: 0.21828000247478485']
['Test loss: 2.1345627307891846', 'Test acc: 0.2264000028371811']


Epoch 4: 


100%|██████████| 1563/1563 [01:24<00:00, 18.44it/s]


['Train loss: 2.124032735824585', 'Train acc: 0.2292799949645996']
['Test loss: 2.098832607269287', 'Test acc: 0.23919999599456787']




In [13]:
run(myMod13,num_epochs=5,load=True)

Epoch 0: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.091501235961914', 'Train acc: 0.23739999532699585']
['Test loss: 2.0699949264526367', 'Test acc: 0.24899999797344208']


Epoch 1: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.0656793117523193', 'Train acc: 0.24422000348567963']
['Test loss: 2.047729015350342', 'Test acc: 0.2563000023365021']


Epoch 2: 


...

['Train loss: 2.0448079109191895', 'Train acc: 0.250900000333786']
['Test loss: 2.029047966003418', 'Test acc: 0.26420000195503235']


Epoch 3: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.0262889862060547', 'Train acc: 0.2581999897956848']
['Test loss: 2.011230230331421', 'Test acc: 0.2705000042915344']


Epoch 4: 


100%|██████████| 1563/1563 [02:21<00:00, 11.01it/s]


['Train loss: 2.009009838104248', 'Train acc: 0.2677200138568878']
['Test loss: 1.9943252801895142', 'Test acc: 0.28200000524520874']




In [17]:
%tensorboard --logdir logs/submission/